# SP API Examples
This notebook is an bare-bone example on how to query Amazon SP API.
Useful for API exploration and debugging purposes.

In [6]:
import boto3
import requests
import json
import os
import zlib
import json
import io
from botocore.exceptions import ClientError

## Configuration

In [5]:
AWS_CRED_PROFILE_NAME = "data-prod"
AWS_SESSION_TOKEN = ""

# config this
account_name = "beco"
account_region = "EU"
marketplace_id = (
    "A1F83G8C2ARO7P"  # developer-docs.amazon.com/sp-api/docs/marketplace-ids
)
base_url = "https://sellingpartnerapi-eu.amazon.com"

account_secret_key = "prod/airbyte/sources/sp_api/" + account_name

## Authentication

In [7]:
def get_secret_by_name(secret_name: str):
    secret_str = _get_secret(secret_name)
    return json.loads(secret_str)


def _get_secret(secret_name: str, region_name: str = "eu-west-1"):
    session = boto3.session.Session(profile_name=AWS_CRED_PROFILE_NAME)
    client = session.client(service_name="secretsmanager", region_name=region_name)

    try:
        get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    except ClientError as e:
        raise e

    return get_secret_value_response["SecretString"]

In [8]:
secret = get_secret_by_name(account_secret_key)

In [ ]:
secret

In [10]:
# Get access token
url = "https://api.amazon.com/auth/o2/token"
client_secret = secret["lwa_client_secret"]
refresh_token = secret["refresh_tokens"][account_region]
client_id = secret["lwa_app_id"]

payload = f"grant_type=refresh_token&refresh_token={refresh_token}&client_id={client_id}&client_secret={client_secret}"
headers = {"Content-Type": "application/x-www-form-urlencoded"}

response = requests.request("POST", url, headers=headers, data=payload)
access_token = response.json()["access_token"]

## GET_STRANDED_INVENTORY_UI_DATA

In [ ]:
# Create report
data_start = "2024-01-01T00:00:00Z"
data_end = "2024-06-18T23:59:59Z"
reportType = "GET_STRANDED_INVENTORY_UI_DATA"
url = f"{base_url}/reports/2021-06-30/reports"

payload = {
    "marketplaceIds": [marketplace_id],
    "reportType": reportType,
    "reportOptions": {
        "promotionStartDateFrom": data_start,
        "promotionStartDateTo": data_end,
    },
}

headers = {
    "Accept": "application/json",
    "x-amz-access-token": access_token,
    "X-Amz-Security-Token": AWS_SESSION_TOKEN,
    "content-type": "application/json; charset=utf-8",
}

response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response.json()

In [ ]:
## get the report
url = f"{base_url}/reports/2021-06-30/reports/{response.json()['reportId']}"

headers = {
    "Accept": "application/json",
    "x-amz-access-token": access_token,
    "X-Amz-Security-Token": AWS_SESSION_TOKEN,
    "content-type": "application/json; charset=utf-8",
}

response = requests.request("get", url, headers=headers)
report = response.json()
report

In [27]:
## get the doc of the report
report_doc_id = report["reportDocumentId"]
url = f"{base_url}/reports/2021-06-30/documents/{report_doc_id}"

headers = {
    "Accept": "application/json",
    "x-amz-access-token": access_token,
    "X-Amz-Security-Token": AWS_SESSION_TOKEN,
    "content-type": "application/json; charset=utf-8",
}

response = requests.request("get", url, headers=headers)

In [ ]:
response.json()

In [ ]:
url = response.json()["url"]
url